## Find location of employees

In [1]:
library("RODBC")

connection<- odbcConnect("Dung")
myData <- sqlQuery(connection, "SELECT
    Address from Employees
 ")
myData

Address
"5631 Rice, OakPark,IL"
"980 Berry ln, Elgin,IL"
"291 Springs, Gary,IL"
"511 Aurora Av, Aurora,IL"
"216 Oak Tree, Geneva,IL"
"111 Green Pl, Elgin,IL"
"100 Rose Pl, Gary,IL"
"145 Berry Ln, Naperville,IL"
"120 Fall Creek, Gary,IL"
"111 Britany Springs,Elgin,IL"


## Find lon and lat of above location

In [9]:
library(jsonlite)
library(dplyr)

# Function to retrieve lon and lat for an address
getLonLat <- function(address) {
  # Create the API URL with the address
  url <- paste0("https://api.geoapify.com/v1/geocode/search?text=", urltools::url_encode(address), "&apiKey=e646e66a03ab4f5b9b4ced370b1f408b")

  # Make the API request and parse the JSON response
  response <- jsonlite::fromJSON(url)
  
  # Check if any features are returned
  if (length(response$features) > 0) {
    # Find the index of the first occurrence of "state_code" = "IL"
    index <- which(response$features$properties$state_code == "IL")[1]
    
    # Check if index exists and extract lon and lat
    if (!is.na(index)) {
      lon <- response$features$geometry$coordinates[[index]][1]
      lat <- response$features$geometry$coordinates[[index]][2]
    } else {
      lon <- NaN
      lat <- NaN
    }
  } else {
    lon <- NaN
    lat <- NaN
  }
  
  return(c(lon, lat))
}

# Create an empty dataframe to store the results
lonlat_df <- data.frame(Address = character(),
                        lon = numeric(),
                        lat = numeric(),
                        stringsAsFactors = FALSE)

# Loop over each address in the myData dataframe
for (i in 1:nrow(myData)) {
  address <- myData$Address[i]
  
  # Retrieve lon and lat for the address
  lonlat <- getLonLat(address)
  
  # Append the result to the lonlat_df dataframe
  lonlat_df <- lonlat_df %>%
    add_row(Address = address, lon = lonlat[1], lat = lonlat[2])
}

# Print the lonlat_df dataframe
print(lonlat_df)

                        Address       lon      lat
1         5631 Rice, OakPark,IL -87.78994 41.88716
2        980 Berry ln, Elgin,IL -88.28110 42.03726
3          291 Springs, Gary,IL -87.88732 41.77956
4      511 Aurora Av, Aurora,IL -88.31100 41.78209
5       216 Oak Tree, Geneva,IL -88.29771 41.88522
6        111 Green Pl, Elgin,IL -88.28110 42.03726
7          100 Rose Pl, Gary,IL -87.85561 41.77059
8   145 Berry Ln, Naperville,IL -88.09061 41.74854
9       120 Fall Creek, Gary,IL -87.85561 41.77059
10 111 Britany Springs,Elgin,IL -88.28110 42.03726


## Find location of Q3

In [10]:
# Address to retrieve lon and lat for
address <- "855 E Golf Rd Arlington Heights, IL 60005 USA"

# Create an empty dataframe to store the result
lonlat_df_2 <- data.frame(Address = character(),
                        lon = numeric(),
                        lat = numeric(),
                        stringsAsFactors = FALSE)

# Retrieve lon and lat for the address
lonlat_2 <- getLonLat(address)

# Add the result to the lonlat_df dataframe
lonlat_df_2 <- lonlat_df_2 %>%
  add_row(Address = address, lon = lonlat_2[1], lat = lonlat_2[2])

# Print the lonlat_df dataframe
print(lonlat_df_2)

                                        Address       lon      lat
1 855 E Golf Rd Arlington Heights, IL 60005 USA -87.97127 42.05043


## Save these df into db

In [11]:
# Create the Employ_loca table in the database
sqlQuery(connection, "CREATE TABLE Employee_Location (
                         Address VARCHAR(100),
                         lon FLOAT,
                         lat FLOAT
                      )")

# Insert the data into the table
sqlSave(connection, lonlat_df, tablename = "Employee_Location", append = TRUE, rownames = FALSE)

character(0)

In [12]:
# Create the Q3_loca table in the database
sqlQuery(connection, "CREATE TABLE Q3_Location (
                         Address VARCHAR(100),
                         lon FLOAT,
                         lat FLOAT
                      )")

# Insert the data into the table
sqlSave(connection, lonlat_df_2, tablename = "Q3_Location", append = TRUE, rownames = FALSE)

character(0)

# Find the distance

In [2]:
myResult <- sqlQuery(connection, "SELECT
    el.Address,
    2 * 3961 * ASIN(SQRT(POWER(SIN((el.Lat - ql.Lat) * PI() / 180 / 2), 2) +
                        COS(el.Lat * PI() / 180) * COS(ql.Lat * PI() / 180) *
                        POWER(SIN((el.Lon - ql.Lon) * PI() / 180 / 2), 2))) AS Distance
FROM
    Employee_Location el
CROSS JOIN
    Q3_Location ql
 ")
myResult

Address,Distance
"5631 Rice, OakPark,IL",14.63819
"980 Berry ln, Elgin,IL",15.93263
"291 Springs, Gary,IL",19.21768
"511 Aurora Av, Aurora,IL",25.48639
"216 Oak Tree, Geneva,IL",20.29776
"111 Green Pl, Elgin,IL",15.93263
"100 Rose Pl, Gary,IL",20.24055
"145 Berry Ln, Naperville,IL",21.75495
"120 Fall Creek, Gary,IL",20.24055
"111 Britany Springs,Elgin,IL",15.93263


In [5]:
# Find the index of the row 
index_min <- which.min(myResult$Distance)
index_max <- which.max(myResult$Distance)
# Get the location and distance 
smallest_location <- myResult$Address[index_min]
smallest_distance <- myResult$Distance[index_min]
max_location <- myResult$Address[index_min]
max_distance <- myResult$Distance[index_min]
# Print 
print(paste("Location min:", smallest_location))
print(paste("Distance min:", smallest_distance))
print(paste("Location max:", max_location))
print(paste("Distance max:", max_distance))

[1] "Location min: 5631 Rice, OakPark,IL"
[1] "Distance min: 14.6381858327869"
[1] "Location max: 5631 Rice, OakPark,IL"
[1] "Distance max: 14.6381858327869"


In [18]:
# Close the database connection
odbcClose(connection)